In [14]:
# =========================
# Cell 0: Imports & config
# =========================

import os
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    roc_auc_score,
)

# Paths (adjust if needed)
POS_FASTA = "Positive_Final.fasta"   # positive = RBP
NEG_FASTA = "Negative_Final.fasta"   # negative = non-RBP
PSSM_DIR  = "pssm_files"             # if/when you add PSSM files
RES_PRED_CSV = "residue_predictions.csv"  # optional PRBR outputs

# Amino acid definitions
AA_ORDER = list("ACDEFGHIKLMNPQRSTVWY")
AA_INDEX = {aa: i for i, aa in enumerate(AA_ORDER)}
N_AA = len(AA_ORDER)

RANDOM_STATE = 42


In [15]:
# =========================
# Cell 1: FASTA utilities
# =========================

def read_fasta(path):
    """
    Minimal FASTA reader.
    Returns dict: {sequence_id: sequence_string}
    """
    seqs = {}
    header = None
    parts = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header is not None:
                    seqs[header] = "".join(parts).replace(" ", "").upper()
                header = line[1:].split()[0]
                parts = []
            else:
                parts.append(line)
    if header is not None:
        seqs[header] = "".join(parts).replace(" ", "").upper()
    return seqs


# Load your dataset
pos_seqs = read_fasta(POS_FASTA)
neg_seqs = read_fasta(NEG_FASTA)

print(f"Loaded {len(pos_seqs)} positive (RBP) sequences")
print(f"Loaded {len(neg_seqs)} negative (non-RBP) sequences")

def build_labeled_df(pos_dict, neg_dict):
    rows = []
    for sid, seq in pos_dict.items():
        rows.append({"id": sid, "sequence": seq, "label": 1})
    for sid, seq in neg_dict.items():
        rows.append({"id": sid, "sequence": seq, "label": 0})
    return pd.DataFrame(rows)

df = build_labeled_df(pos_seqs, neg_seqs)
print(df.head())
print(df["label"].value_counts())


Loaded 3741 positive (RBP) sequences
Loaded 3712 negative (non-RBP) sequences
                          id  \
0  sp|A0A0A7HFE1|CAS10_STRTR   
1  sp|A0A0A7HIX6|CSM6A_STRTR   
2   sp|A0A0B4KGY6|NOVA_DROME   
3   sp|A0A0D1DWZ5|RRM4_MYCMD   
4  sp|A0A0F6B5X4|TACT3_SALT1   

                                            sequence  label  
0  MKKEKIDLFYGALLHDIGKVIQRATGERKKHALVGADWFDEIADNQ...      1  
1  MKILISAVGTTDPISNNHDAALLHIARNYRPDKIVLVYSQEMMVKQ...      1  
2  MESIMKVAMDKAAEQLIQQFGFDYLQQQLQLQHQNQHNSSPQQPQH...      1  
3  MSDSIYAPHNKHKLEAARAADAAADDAATVSALVEPTDSTAQASHA...      1  
4  MMFTDWHEAAIGKTHNRMNFDCGDADLNQFLQRHARQNHEKGTTKT...      1  
label
1    3741
0    3712
Name: count, dtype: int64


In [16]:
# =========================
# Cell 2: AAC features
# =========================

def compute_aac(seq, aa_order=AA_ORDER):
    """
    Amino Acid Composition (AAC):
    p_i = l_i / L, where l_i is count of AA i and L is sequence length.
    Returns a 20-d numpy array in the fixed AA_ORDER.
    """
    counts = np.zeros(len(aa_order), dtype=float)
    L = 0
    for aa in seq:
        if aa in AA_INDEX:
            counts[AA_INDEX[aa]] += 1
            L += 1
    if L > 0:
        counts /= L
    return counts

# Quick sanity check
test_aac = compute_aac(df["sequence"].iloc[0])
print("AAC length:", len(test_aac))
print("AAC sum:", test_aac.sum())


AAC length: 20
AAC sum: 1.0


In [17]:
# =======================================
# Cell 3: EIPP feature skeleton (optional)
# =======================================

# TODO: replace 0.0 with real values from the paper / references.
PHYSICOCHEMICAL_PROPERTIES = {
    "pka_amino":    {aa: 0.0 for aa in AA_ORDER},
    "pka_carboxyl": {aa: 0.0 for aa in AA_ORDER},
    "mass":         {aa: 0.0 for aa in AA_ORDER},
    "eiip":         {aa: 0.0 for aa in AA_ORDER},
    "lone_pairs":   {aa: 0.0 for aa in AA_ORDER},
    "wiener_index": {aa: 0.0 for aa in AA_ORDER},
}

def normalize_property_vector(values_dict):
    vals = np.array([values_dict[aa] for aa in AA_ORDER], dtype=float)
    vmin = vals.min()
    vmax = vals.max()
    if vmax == vmin:
        return np.zeros_like(vals)
    return (vals - vmin) / (vmax - vmin)

def load_pssm_for_sequence(seq_id, pssm_dir=PSSM_DIR):
    """
    TODO: parse your real PSSM files here.
    This should return a numpy array (L, 20) of raw PSSM scores.
    """
    path = os.path.join(pssm_dir, f"{seq_id}.pssm")
    if not os.path.exists(path):
        raise FileNotFoundError(f"PSSM not found for {seq_id}: {path}")

    rows = []
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split()
            # Adapt this parser to your PSSM format
            if len(parts) >= 22 and parts[0].isdigit():
                scores = [float(x) for x in parts[2:22]]
                rows.append(scores)
    if not rows:
        raise ValueError(f"No PSSM rows parsed for {seq_id}")
    return np.array(rows, dtype=float)

def normalize_pssm(pssm):
    """
    Sigmoid normalization: f(x) = 1 / (1 + exp(-x)), as in the paper.
    """
    return 1.0 / (1.0 + np.exp(-pssm))

def compute_eipp_features(seq_id, seq, pssm_dir=PSSM_DIR):
    """
    120-dim EIPP feature: 6 properties x 20 amino acid types.
    Follows the description in the PRBP paper.
    """
    pssm_raw = load_pssm_for_sequence(seq_id, pssm_dir=pssm_dir)
    pssm_norm = normalize_pssm(pssm_raw)  # (L, 20)

    if pssm_norm.shape[0] != len(seq):
        raise ValueError(f"PSSM length ({pssm_norm.shape[0]}) != sequence length ({len(seq)}) for {seq_id}")

    normalized_props = {
        name: normalize_property_vector(values)
        for name, values in PHYSICOCHEMICAL_PROPERTIES.items()
    }

    eipp_vec = []
    for prop_name, da in normalized_props.items():          # 6 properties
        for aa_k in AA_ORDER:                              # 20 amino acids
            positions = [i for i, aa in enumerate(seq) if aa == aa_k]
            if not positions:
                eipp_vec.append(0.0)
                continue
            sub_pssm = pssm_norm[positions, :]             # (#pos, 20)
            f_k = sub_pssm.mean(axis=0)                    # average over positions
            contrib = np.sum(np.sqrt(da) * f_k)            # Eq.(5) structure
            eipp_vec.append(contrib)
    return np.array(eipp_vec, dtype=float)


In [18]:
# ===================================
# Cell 4: Extract features for all seq
# ===================================

def extract_features(df, use_eipp=False, pssm_dir=PSSM_DIR):
    X_list = []
    y_list = []
    ids = []

    for _, row in df.iterrows():
        seq_id = row["id"]
        seq = row["sequence"]
        label = row["label"]

        aac = compute_aac(seq)  # 20 dims
        feats = [aac]

        if use_eipp:
            eipp = compute_eipp_features(seq_id, seq, pssm_dir=pssm_dir)  # 120 dims
            feats.append(eipp)

        X_list.append(np.concatenate(feats))
        y_list.append(label)
        ids.append(seq_id)

    X = np.vstack(X_list)
    y = np.array(y_list, dtype=int)
    ids = np.array(ids)
    return X, y, ids

# Start with AAC-only (no PSSM needed)
X, y, ids = extract_features(df, use_eipp=False)
print("Feature matrix shape:", X.shape)


Feature matrix shape: (7453, 20)


In [19]:
# ======================================
# Cell 5: Train-test split + RF training
# ======================================

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE,
)

rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print(f"Test Accuracy:  {acc:.4f}")
print(f"Test Precision: {prec:.4f}")
print(f"Test Recall:    {rec:.4f}")
print(f"Test F1-score:  {f1:.4f}")
print(f"Test MCC:       {mcc:.4f}")
print(f"Test AUC:       {auc:.4f}")


Test Accuracy:  0.7311
Test Precision: 0.7245
Test Recall:    0.7487
Test F1-score:  0.7364
Test MCC:       0.4623
Test AUC:       0.8096


In [20]:
# ======================================
# Cell 6: 5-fold cross-validation (RF)
# ======================================

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc",
}

cv_results = cross_validate(
    rf,
    X,
    y,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
)

for metric, values in cv_results.items():
    if metric.startswith("test_"):
        name = metric.replace("test_", "")
        print(f"{name:10s}: mean={values.mean():.4f}, std={values.std():.4f}")


accuracy  : mean=0.7355, std=0.0042
precision : mean=0.7296, std=0.0092
recall    : mean=0.7522, std=0.0153
f1        : mean=0.7406, std=0.0048
roc_auc   : mean=0.8159, std=0.0053


In [21]:
# ======================================
# Cell 7: Predict 5 random sequences
# ======================================

# Predictions for full dataset
all_probs = rf.predict_proba(X)[:, 1]
all_preds = rf.predict(X)

results = df.copy()
results["pred_label"] = all_preds
results["pred_proba_RBP"] = all_probs

sample5 = results.sample(n=5, random_state=RANDOM_STATE)

for _, row in sample5.iterrows():
    print(f"ID:           {row['id']}")
    print(f"True label:   {row['label']}   (1 = RBP, 0 = non-RBP)")
    print(f"Predicted:    {row['pred_label']}")
    print(f"P(RBP | seq): {row['pred_proba_RBP']:.4f}")
    seq = row["sequence"]
    print(f"Seq (first 60 aa): {seq[:60]}{'...' if len(seq) > 60 else ''}")
    print("-" * 60)


ID:           sp|O48398|GP42_BPSP1
True label:   0   (1 = RBP, 0 = non-RBP)
Predicted:    0
P(RBP | seq): 0.2140
Seq (first 60 aa): MRKFVTTLTASPRNKKVGNHRLEISPFVSLRRYYYFNTAICIENPVTREFAIDDSYGSLS...
------------------------------------------------------------
ID:           sp|Q6AQA5|RLMKL_DESPS
True label:   1   (1 = RBP, 0 = non-RBP)
Predicted:    1
P(RBP | seq): 0.7200
Seq (first 60 aa): MCDKKDVSPQKDQYTFLANCALGLEELIEAEIKGFSGVEVELGKGTVQWQGSLETGYRAC...
------------------------------------------------------------
ID:           sp|P64578|HIGB_ECOLI
True label:   1   (1 = RBP, 0 = non-RBP)
Predicted:    1
P(RBP | seq): 0.8480
Seq (first 60 aa): MHLITQKALKDAAEKYPQHKTELVALGNTIAKGYFKKPESLKAVFPSLDNFKYLDKHYVF...
------------------------------------------------------------
ID:           sp|Q976I5|RL40_SULTO
True label:   0   (1 = RBP, 0 = non-RBP)
Predicted:    0
P(RBP | seq): 0.2080
Seq (first 60 aa): MPLTDPVKLQIVQQRIFLKKVCRDCGALNSVRATKCRRCHSKNLRPKKKELPAKKG
---------------------------------------

In [22]:
# ======================================
# Cell 8: RM(1) computation
# ======================================

def compute_rm1(ri_values, N):
    """
    RM(1) = sum RI(i) / (10 * N), as defined in the PRBP paper.
    ri_values: list or array of RI(i) for all predicted binding residues
    N: protein length
    """
    ri_values = np.asarray(ri_values, dtype=float)
    if N <= 0 or ri_values.size == 0:
        return 0.0
    return ri_values.sum() / (10.0 * N)

# Thresholds used in the paper (for RM(1))
RM1_UPPER = 0.4
RM1_LOWER = 0.02


In [23]:
# ======================================
# Cell 9: Load PRBR-like residue predictions (optional)
# ======================================

def load_residue_predictions(csv_path=RES_PRED_CSV):
    """
    Load a CSV file with residue-level predictions.
    Expected columns: id, position, RI, is_binding
    """
    if not os.path.exists(csv_path):
        print(f"[INFO] Residue prediction CSV '{csv_path}' not found. "
              f"RM(1)-based rules will be skipped.")
        return None
    res_df = pd.read_csv(csv_path)
    required_cols = {"id", "position", "RI", "is_binding"}
    if not required_cols.issubset(set(res_df.columns)):
        raise ValueError(f"Residue CSV must contain columns: {required_cols}")
    return res_df

residue_predictions_df = None


In [24]:
# ======================================
# Cell 10: Helper to get residue info
# ======================================

def get_binding_residues_for_protein(seq_id, residue_df):
    """
    Returns (binding_positions, ri_values) for a given protein id.
    binding_positions: list of positions (int)
    ri_values: list of RI(i) for those positions
    """
    if residue_df is None:
        return [], []

    sub = residue_df[(residue_df["id"] == seq_id) & (residue_df["is_binding"] == 1)]
    if sub.empty:
        return [], []
    # assuming positions are 0-based; if 1-based, subtract 1
    positions = sub["position"].astype(int).tolist()
    ri_values = sub["RI"].astype(float).tolist()
    return positions, ri_values


In [25]:
# ======================================
# Cell 11: PRBP-style prediction for one protein
# ======================================

def prbp_predict_single(seq_id, sequence, rf_model, residue_df=None,
                        use_eipp=False, pssm_dir=PSSM_DIR,
                        prob_threshold=0.5):
    """
    PRBP-style molecular prediction for a single protein:
      1) If residue_df is available:
         - get binding residues & RI
         - compute RM(1)
         - if RM(1) > 0.4 -> RBP
         - if RM(1) < 0.02 -> non-RBP
      2) Otherwise (or in between):
         - use RF classifier with AAC (+ optional EIPP) features.

    Returns:
      label (0/1),
      details dict (RM1, decision_source, RF probability, etc.)
    """
    N = len(sequence)
    # Step 1: residue-level info
    binding_positions, ri_values = get_binding_residues_for_protein(seq_id, residue_df)

    rm1 = compute_rm1(ri_values, N)

    # Decision purely from RM(1), if residue info is available
    if residue_df is not None:
        if rm1 > RM1_UPPER:
            return 1, {"RM1": rm1, "decision_source": "RM1_high"}
        if rm1 < RM1_LOWER:
            return 0, {"RM1": rm1, "decision_source": "RM1_low"}

    # Step 2: RF classification
    temp_df = pd.DataFrame([{"id": seq_id, "sequence": sequence, "label": 0}])
    X_single, _, _ = extract_features(temp_df, use_eipp=use_eipp, pssm_dir=pssm_dir)
    proba_rbp = rf_model.predict_proba(X_single)[0, 1]
    pred_label = int(proba_rbp >= prob_threshold)

    details = {
        "RM1": rm1,
        "decision_source": "RF",
        "rf_proba_RBP": float(proba_rbp),
    }
    return pred_label, details


In [26]:
# ======================================
# Cell 12: PRBP-style prediction on 5 random sequences
# ======================================

sample5 = df.sample(n=5, random_state=RANDOM_STATE)

for _, row in sample5.iterrows():
    seq_id = row["id"]
    seq = row["sequence"]
    true_label = row["label"]

    pred_label, info = prbp_predict_single(
        seq_id,
        seq,
        rf_model=rf,
        residue_df=residue_predictions_df,
        use_eipp=False,          # switch to True after adding EIPP
        prob_threshold=0.5,
    )

    print(f"ID:            {seq_id}")
    print(f"True label:    {true_label}")
    print(f"Predicted:     {pred_label}")
    print(f"RM(1):         {info.get('RM1', 0.0):.4f}")
    if "rf_proba_RBP" in info:
        print(f"P(RBP | RF):   {info['rf_proba_RBP']:.4f}")
    print(f"Decision via:  {info['decision_source']}")
    print(f"Seq (first 60): {seq[:60]}{'...' if len(seq) > 60 else ''}")
    print("-" * 60)


ID:            sp|O48398|GP42_BPSP1
True label:    0
Predicted:     0
RM(1):         0.0000
P(RBP | RF):   0.2140
Decision via:  RF
Seq (first 60): MRKFVTTLTASPRNKKVGNHRLEISPFVSLRRYYYFNTAICIENPVTREFAIDDSYGSLS...
------------------------------------------------------------
ID:            sp|Q6AQA5|RLMKL_DESPS
True label:    1
Predicted:     1
RM(1):         0.0000
P(RBP | RF):   0.7200
Decision via:  RF
Seq (first 60): MCDKKDVSPQKDQYTFLANCALGLEELIEAEIKGFSGVEVELGKGTVQWQGSLETGYRAC...
------------------------------------------------------------
ID:            sp|P64578|HIGB_ECOLI
True label:    1
Predicted:     1
RM(1):         0.0000
P(RBP | RF):   0.8480
Decision via:  RF
Seq (first 60): MHLITQKALKDAAEKYPQHKTELVALGNTIAKGYFKKPESLKAVFPSLDNFKYLDKHYVF...
------------------------------------------------------------
ID:            sp|Q976I5|RL40_SULTO
True label:    0
Predicted:     0
RM(1):         0.0000
P(RBP | RF):   0.2080
Decision via:  RF
Seq (first 60): MPLTDPVKLQIVQQRIFLKKVCRDCGALNSVR